<a href="https://colab.research.google.com/github/kevkn2/projects-mine/blob/main/test_capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unrar x "/content/TRAIN.rar" "/content/"
!unrar x "/content/VAL.rar" "/content/"


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/TRAIN.rar

Creating    /content/TRAIN                                            OK
Creating    /content/TRAIN/Kain Tenun Toraja                          OK
Extracting  /content/TRAIN/Kain Tenun Toraja/toraja1.jpg                   0%  1%  OK 
Extracting  /content/TRAIN/Kain Tenun Toraja/toraja10.jpg                  2%  3%  4%  OK 
Extracting  /content/TRAIN/Kain Tenun Toraja/toraja2.jpg                   8%  OK 
Extracting  /content/TRAIN/Kain Tenun Toraja/toraja3.jpg                  15%  OK 
Extracting  /content/TRAIN/Kain Tenun Toraja/toraja5.jpg                  16% 17% 18% 19%  OK 
Extracting  /content/TRAIN/Kain Tenun Toraja/toraja6.jpg                  20% 21% 22% 23% 24% 25% 26% 27%  OK 
Extracting  /content/TRAIN/Kain Tenun Toraja/toraja8.jpg                  28%  OK 
E

In [11]:
import tensorflow as tf
import numpy as np
import os
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub

train_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                   rotation_range=45,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
        '/content/TRAIN',
        target_size=(224,224),
        batch_size=2,
        class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
        '/content/VAL',
        target_size=(224,224),
        batch_size=2,
        class_mode='categorical')

Found 24 images belonging to 3 classes.
Found 9 images belonging to 3 classes.


In [20]:
model_transfer = tf.keras.applications.resnet50.ResNet50(
    input_shape=(224,224,3)
)

In [14]:
x = tf.keras.layers.Dense(512, activation="relu")(model_transfer.output)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(3, activation="softmax")(x)
model = tf.keras.Model(model_transfer.input, x)

#model = tf.keras.models.Sequential([
#    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300,300,3)),
#    tf.keras.layers.MaxPooling2D(2,2),
#    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
#    tf.keras.layers.MaxPooling2D(2,2),
#    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#    tf.keras.layers.MaxPooling2D(2,2),
#    tf.keras.layers.Flatten(),
#    tf.keras.layers.Dense(512, activation = 'relu'),
#    tf.keras.layers.Dense(3, activation = 'softmax')
#])

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              metrics=['accuracy'])

history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=15,
                    verbose=1)

Epoch 1/15
12/12 [==============================] - 13s 178ms/step - loss: 1.0949 - accuracy: 0.2083 - val_loss: 1.1047 - val_accuracy: 0.2222
Epoch 2/15
12/12 [==============================] - 1s 88ms/step - loss: 1.0905 - accuracy: 0.4583 - val_loss: 1.1050 - val_accuracy: 0.2222
Epoch 3/15
12/12 [==============================] - 1s 90ms/step - loss: 1.0805 - accuracy: 0.5000 - val_loss: 1.1069 - val_accuracy: 0.2222
Epoch 4/15
12/12 [==============================] - 1s 90ms/step - loss: 1.0661 - accuracy: 0.6667 - val_loss: 1.1074 - val_accuracy: 0.2222
Epoch 5/15
12/12 [==============================] - 1s 89ms/step - loss: 1.0690 - accuracy: 0.5417 - val_loss: 1.1112 - val_accuracy: 0.2222
Epoch 6/15
12/12 [==============================] - 1s 89ms/step - loss: 1.0565 - accuracy: 0.6667 - val_loss: 1.1058 - val_accuracy: 0.1111
Epoch 7/15
12/12 [==============================] - 1s 89ms/step - loss: 1.0531 - accuracy: 0.6667 - val_loss: 1.1076 - val_accuracy: 0.2222
Epoch 8/15


In [19]:
img = image.load_img('/content/toraja7.jpg', target_size=(224,224))
x = image.img_to_array(img)
x = x/255
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images, batch_size=2)
for result in classes:
  print(result)


[0.29477593 0.3807018  0.32452226]
